In [66]:
import numpy as np
import scipy as sy
from scipy.optimize import root
from matplotlib import pyplot as plt

Problem 1

In [67]:
def equations1(v, g):
    '''
    Function which defines and output a list of functions defining the solution to the model
    Inputs: 
        v: vector of inputs to the function
    Output:
        rv: list of functions
    '''
    a_1_1, a_2_1, a_1_2, a_2_2, r_1, r_2 = v
    rv = []
    f_a_1_1 = (-r_1*(1- (r_1 * a_1_1) + (r_2 * a_2_1))** -g + 
            .25 * ( (1+a_1_1+a_2_1)**-g + (2+a_1_1+a_2_1)**-g + 
            (1+a_1_1+1.5*a_2_1)**-g  + (2+a_1_1+1.5*a_2_1)**-g ) ) 
    rv.append(f_a_1_1)
    
    f_a_2_1 = (-r_2*(1- (r_1 * a_1_1) + (r_2 * a_2_1))**-g + 
            .25 * ( (1+a_1_1+a_2_1)**-g + (2+a_1_1+a_2_1)**-g + 
            1.5*(1+a_1_1+1.5*a_2_1)**-g  + 1.5*(2+a_1_1+1.5*a_2_1)**-g ) )
    rv.append(f_a_2_1)
    
    f_a_1_2 = (-r_1*(1- (r_1 * a_1_2) + (r_2 * a_2_2))** -g + 
            .25 * ( (3+a_1_2+a_2_2)**-g + (1+a_1_2+a_2_2)**-g + 
            (3+a_1_2+1.5*a_2_2)**-g  + (1+a_1_2+1.5*a_2_2)**-g ) )
    rv.append(f_a_1_2)
    
    f_a_2_2 = (-r_2*(1- (r_1 * a_1_2) + (r_2 * a_2_2))** -g + 
            .25 * ( (3+a_1_2+a_2_2)**-g + (1+a_1_2+a_2_2)**-g + 
            1.5*(3+a_1_2+1.5*a_2_2)**-g  + 1.5*(1+a_1_2+1.5*a_2_2)**-g ) ) 
    rv.append(f_a_2_2)
    rv.append(a_1_1 + a_1_2)
    rv.append(a_2_1 + a_2_2)
    return rv 

def solver1(g):
    rv =  root(equations1, [.5, .5, .5, .5, .5, .5], method = 'hybr', args=(g)).x
    return rv

class Solution1:
    def __init__(self, arr):
        self.A11 = arr[0]
        self.A21 = arr[1]
        self.A12 = arr[2]
        self.A22 = arr[3]
        self.R1 = arr[4]
        self.R2 = arr[5]
     
    def __str__(self):
        rv = []
        rv.append('{:<20}{}'.format('H1, Asset 1:', self.A11))
        rv.append('{:<20}{}'.format('H1, Asset 2:', self.A21))
        rv.append('{:<20}{}'.format('H2, Asset 1:', self.A12))
        rv.append('{:<20}{}'.format('H2, Asset 2:', self.A22))
        rv.append('{:<20}{}'.format('Asset 1, Price:', self.R1))
        rv.append('{:<20}{}'.format('Asset 2, Price:', self.R2))
        return '\n'.join(rv)
    

for g in [2, 4, 8, 166]:
    print("Gamma: {}".format(g))
    tmp = Solution1(solver1(g))
    print(tmp)
    if g < 100:
        print('\n \n')
    

Gamma: 2
H1, Asset 1:        0.019540665546350387
H1, Asset 2:        -5.997142761502882e-12
H2, Asset 1:        -0.019540665546350387
H2, Asset 2:        5.997142751453827e-12
Asset 1, Price:     0.5897776558197745
Asset 2, Price:     0.7372220697751221

 

Gamma: 4
H1, Asset 1:        0.004029823681645771
H1, Asset 2:        -4.520386315367643e-11
H2, Asset 1:        -0.00402982368164577
H2, Asset 2:        4.5203863368132257e-11
Asset 1, Price:     0.5186613028541645
Asset 2, Price:     0.6483266285778475

 

Gamma: 8
H1, Asset 1:        0.0001560941529999874
H1, Asset 2:        4.338013916463119e-13
H2, Asset 1:        -0.0001560941529999874
H2, Asset 2:        -4.3380139189834673e-13
Asset 1, Price:     0.5010144010354313
Asset 2, Price:     0.626268001293289

 

Gamma: 166
H1, Asset 1:        6.502492716803548
H1, Asset 2:        4.074527337326263
H2, Asset 1:        -6.502492716803548
H2, Asset 2:        -4.074527337326263
Asset 1, Price:     -2.524271258962357e-24
Asset 2, Pric

Problem 3

In [104]:
# Init Params
beta = .9 
pi = .5 
s_grid = [1, 2]
eps = .0001
max_iterations = 500

def objective_3(k, k1, V, state):
    '''
    Objective Function Value
    Given a specific k_index which will refer to a 
    '''
    val = np.zeros_like(k1)
    if state == 1:
        c = (((.9 * k) ** 0.3) + (.3*k) - k1)
    else:
        c = (((1.1 * k) ** 0.3) + (.9*k) - k1)
    val[c <= 0] = -999999
    val[c > 0] = np.log(c[c > 0]) + beta * V[c > 0] 
    return val

def ramsey_3(grid_size):
    k_grid = np.linspace(.001, 10, grid_size)     # Array of Admissable capital levels 
    ## Upper Case V denotes the average of the two states, lower case v denotes state specific value
    V = np.zeros_like(k_grid)                     # (1, grid_size) dim vector of the value functions 
    v = np.zeros([len(k_grid), len(s_grid)])      # (2, grid_size) array of the value functions by state
    diff_list = []                                # diff_list
    pol = np.zeros_like(v)                        # (2, grid_size) array of policy functions by state 
    plot_int = 50                                 # Plotting Frequency
    
    for itera in range(max_iterations):
        V_new = np.zeros_like(V)
        v_new = np.zeros_like(v)
        for i, k in enumerate(k_grid):
            for j, s in enumerate(s_grid):
                val = objective_3(k, k_grid, V, s)
                v_new[i, j] = np.amax(val)
                pol[i, j] = k_grid[np.argmax(val)]
            V_new[i] = pi*v_new[i, 0] + pi*v_new[i, 1]
            
            
        diff = np.amax(abs(v_new - v))
        diff_list.append(diff)
        V = np.copy(V_new)
        v = np.copy(v_new)
        if (diff < eps):
            print('Converged after iteration {}'.format(itera +1))
            plt.figure()
            plt.plot(k_grid, v[:, 0], label = "Bad Shock (1)")
            plt.plot(k_grid, v[:, 1], label = "Good Shock (2)")
            plt.legend(loc="lower right")
            plt.xlabel('k')
            plt.ylabel('V(k)')
            plt.title('Value function after convergence')
            plt.show();
            
            
            plt.figure()
            plt.plot(np.linspace(.001, 10, grid_size), pol[:,0], label = "Bad Shock (1)")
            plt.plot(np.linspace(.001, 10, grid_size), pol[:,1], label = "Good Shock (2)")
            plt.legend(loc="lower right")
            plt.xlabel('k')
            plt.ylabel('k\'')
            plt.title('Value function after convergence')
            plt.show();
            break
            
        elif (itera % plot_int == 1 ):
            plt.figure()
            plt.plot(np.arange(1, itera+2), np.log10(np.array(diff_list)))
            plt.xlabel('Iteration')
            plt.ylabel('Maximum update [log10]')
            plt.show();
            
ramsey_3(50)
ramsey_3(500)
        